In [144]:
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import numpy as np
import re

from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.family'] = 'Malgun Gothic'

from tqdm import tqdm
tqdm.pandas()

In [145]:
consumer = pd.read_csv("../Data/Preprocessed/사용자분석.csv")
charge1 = pd.read_csv("../Data/Preprocessed/충전데이터(2304).csv")
charge2 = pd.read_csv("../Data/Preprocessed/충전데이터(2312).csv")

## 사용자데이터

In [146]:
consumer

,사용자코드,연도,분기,월,날짜,요일,시간대명,성별,연령대,나이,...,시,도,읍/면,가맹점명,골목상권분류,대분류,중분류,소분류,가맹점 주소,가맹가입일자
0,차기순01041235434,2019년,4,12월,20191226,목,12시~17:59분,여자,40대,40.0,...,부여군,충남,부여읍,문광사,기타,기타,기타,기타,부여읍,2019-11-05
1,차기순01041235434,2020년,1,1월,20200107,화,18시~23시59분,여자,40대,41.0,...,부여군,충남,부여읍,문광사,기타,기타,기타,기타,부여읍,2019-11-05
2,차기순01041235434,2020년,2,5월,20200514,목,12시~17:59분,여자,40대,41.0,...,부여군,충남,부여읍,문광사,기타,기타,기타,기타,부여읍,2019-11-05
3,차기순01041235434,2020년,2,6월,20200626,금,12시~17:59분,여자,40대,41.0,...,부여군,충남,부여읍,문광사,기타,기타,기타,기타,부여읍,2019-11-05
4,차기순01041235434,2020년,3,7월,20200707,화,06시~11시59분,여자,40대,41.0,...,부여군,충남,부여읍,문광사,기타,기타,기타,기타,부여읍,2019-11-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10569213,오경순01094315714,2022년,4,12월,20221209,금,12시~17:59분,여자,60대,60.0,...,논산시,충남,NaN,동부여농협주유소,대형,자동차,주유소/충전소,주유소,초촌면,2020-01-02
10569214,오경순01094315714,2022년,4,12월,20221224,토,06시~11시59분,여자,60대,60.0,...,논산시,충남,NaN,동부여농협주유소,대형,자동차,주유소/충전소,주유소,초촌면,2020-01-02
10569215,오경순01094315714,2023년,1,2월,20230224,금,12시~17:59분,여자,60대,61.0,...,논산시,충남,NaN,동부여농협주유소,대형,자동차,주유소/충전소,주유소,초촌면,2020-01-02
10569216,오경순01094315714,2023년,1,2월,20230224,금,12시~17:59분,여자,60대,61.0,...,논산시,충남,NaN,동부여농협주유소,대형,자동차,주유소/충전소,주유소,초촌면,2020-01-02


In [147]:
consumer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10569218 entries, 0 to 10569217
Data columns (total 24 columns):
 #   Column      Dtype  
---  ------      -----  
 0   사용자코드       object 
 1   연도          object 
 2   분기          int64  
 3   월           object 
 4   날짜          int64  
 5   요일          object 
 6   시간대명        object 
 7   성별          object 
 8   연령대         object 
 9   나이          float64
 10  사용자 주소      object 
 11  가맹점 사업자 번호  int64  
 12  가맹점 사업자 업종  object 
 13  사용금액        int64  
 14  시           object 
 15  도           object 
 16  읍/면         object 
 17  가맹점명        object 
 18  골목상권분류      object 
 19  대분류         object 
 20  중분류         object 
 21  소분류         object 
 22  가맹점 주소      object 
 23  가맹가입일자      object 
dtypes: float64(1), int64(4), object(19)
memory usage: 1.9+ GB


In [149]:
consumer.isna().sum()
##고객 군집화에서 사용금액이 중요하기 때문에 NA 제거하지 말기 

사용자코드              0
연도                 0
분기                 0
월                  0
날짜                 0
요일                 0
시간대명               0
성별                 0
연령대                0
나이                 0
사용자 주소             0
가맹점 사업자 번호         0
가맹점 사업자 업종         0
사용금액               0
시                  0
도                  0
읍/면           168047
가맹점명               0
골목상권분류             0
대분류                0
중분류                0
소분류                0
가맹점 주소             0
가맹가입일자             0
dtype: int64

In [150]:
# #가맹점 주소 잘못된 거 처리 
# ##부여읍 아닌 것들 
# 가맹점_오류 = ['천호대로', '능동로', '신당2길', '백일헌로']
# consumer = consumer.loc[~consumer['가맹점 주소'].isin(가맹점_오류)]

# ##가맹점 주소 대체 List
# 부여읍 = ['성왕로173번길', '정림로', '부여로', '중앙로13번길', '중앙로', '사비로72번길', '사비로', '사비로100번길', '석탑로', '성왕로328번길', '중앙로5번길','성왕로','신기정로','궁남로', '중앙로5번길', '부장대로', '흑천로', '왕릉로', '신동엽길', '성왕로161번길']
# 구룡면 = ['충의로', '흥수로']
# 규암면 = ['자온로', '수북로41번길', '백제문로304번길', '규아면', '반산1구', '수북로', '충절로2123번길','충절로']
# 임천면 = ['성흥로', '성흥로97번길']
# 장암면 = ['의자로1235번길', '의자로', '장암로']
# 석성면 = ['선사로']
# 홍산면 = ['홍산시장로']
# 양화면 = ['입포로48번길']

# ##가맹점 주소 대체 데이터 대체 완료 코드
# consumer.loc[(consumer['가맹점 주소'].isin(부여읍)),['가맹점 주소']] = '부여읍'
# consumer.loc[(consumer['가맹점 주소'].isin(구룡면)),['가맹점 주소']] = '구룡면'
# consumer.loc[(consumer['가맹점 주소'].isin(규암면)),['가맹점 주소']] = '규암면'
# consumer.loc[(consumer['가맹점 주소'].isin(임천면)),['가맹점 주소']] = '임천면'
# consumer.loc[(consumer['가맹점 주소'].isin(장암면)),['가맹점 주소']] = '장암면'
# consumer.loc[(consumer['가맹점 주소'].isin(석성면)),['가맹점 주소']] = '석성면'
# consumer.loc[(consumer['가맹점 주소'].isin(홍산면)),['가맹점 주소']] = '홍산면'
# consumer.loc[(consumer['가맹점 주소'].isin(양화면)),['가맹점 주소']] = '양화면'
# print(consumer['가맹점 주소'].unique())

# ##NA 데이터 삭제 코드
# # consumer = consumer[~consumer['가맹점 주소'].isna()]
# # consumer['가맹점 주소'].unique()

In [151]:
#부여군에서 [부여읍, 규암면, 홍산면, 기타]로 분류 
print(consumer.loc[consumer['시'] == '부여군']['읍/면'].unique())
print(consumer['가맹점 주소'].unique())

['부여읍' '장암면' '규암면' '은산면' '초촌면' '외산면' '구룡면' '남면' '내산면' '석성면' '홍산면' '옥산면'
 '양화면' '임천면' '충화면' '세도면']
['부여읍' '은산면' '규암면' '홍산면' '임천면' '외산면' '구룡면' '석성면' '내산면' '세도면' '초촌면' '남면'
 '장암면' '양화면' '옥산면' '충화면']


In [152]:
def location(data):
    values = ['부여읍', '규암면', '홍산면']
    condition1 = [
        (data['읍/면'] == '부여읍'),
        (data['읍/면'] == '규암면'),
        (data['읍/면'] == '홍산면')
    ] 
    condition2 = [
        (data['가맹점 주소'] == '부여읍'),
        (data['가맹점 주소'] == '규암면'),
        (data['가맹점 주소'] == '홍산면')
    ]
    data.loc[:, '읍/면'] = np.select(condition1, values, default='기타')
    data.loc[:, '가맹점 주소'] = np.select(condition2, values, default='기타')
    return data

buyeo = consumer.loc[consumer['시'] == '부여군']
consumer = location(buyeo)
print(consumer['읍/면'].unique())
print(consumer['가맹점 주소'].unique())

['부여읍' '기타' '규암면' '홍산면']
['부여읍' '기타' '규암면' '홍산면']


In [153]:
#날짜 형태로 변환 
def convert_date(data):
    data.loc[:, '연도'] = data['연도'].str.replace('년', '')
    data = data.drop('월', axis=1)
    data['날짜'] = pd.to_datetime(data['날짜'], format='%Y%m%d')
    data['연월'] = data['날짜'].dt.strftime('%Y-%m')
    return data

consumer = convert_date(consumer)

In [154]:
#연령대 처리
tmp = consumer.groupby(['사용자코드'])['연령대'].apply(lambda x : sorted(x)[0]).reset_index()
tmp = tmp.rename(columns = {'연령대' : '신규_연령대'})
consumer = consumer.merge(tmp, on = ['사용자코드'], how = 'left')
consumer = consumer.drop(columns = ['연령대']).rename(columns = {'신규_연령대' : '연령대'})

In [155]:
#연도별 데이터프레임 나누기 
date_ranges = [('2020-11-01', '2021-10-31'), ('2021-11-01', '2022-10-31'), ('2022-11-01', '2023-10-31')]
year_dict = {}
for idx, (start_date, end_date) in enumerate(date_ranges):
    start_date_dt = pd.to_datetime(start_date)
    end_date_dt = pd.to_datetime(end_date)

    data_filt = consumer[(consumer['날짜'] >= start_date_dt) & (consumer['날짜'] <= end_date_dt)]
    year_dict[f"df_{idx}"] = data_filt

for key, value in year_dict.items():
    globals()[f'{key}'] = value 
#df_0, df_1, df_2 

#### 집계

- 연도별 데이터: df_0, df_1, df_2

In [156]:
##총 사용금액, 1회당 평균 사용금액 
def spend(df):
    a = df.groupby(['사용자코드'])['사용금액'].sum().reset_index()
    a = a.rename(columns={'사용금액': '총 사용금액'})
    b = df.groupby(['사용자코드'])['사용금액'].count().reset_index()
    b = b.rename(columns={'사용금액': '사용빈도'})
    c = pd.merge(a, b, on='사용자코드')
    # c['1회당 평균 사용금액'] = round(c['총 사용금액'] / c['사용빈도'], 2)
    d = df.groupby(['사용자코드'])['사용금액'].mean().apply(lambda x : round(x, 4)).reset_index()
    d = d.rename(columns = {'사용금액' : '1회당 평균 사용금액'})
    c = pd.merge(c, d, on = '사용자코드', how = 'left')
    c = c[['사용자코드', '총 사용금액', '사용빈도', '1회당 평균 사용금액']]
    return c 

In [157]:
##다른지역소비비율 
def diff_region(df):
    total_sum = df.groupby(['사용자코드'])['사용금액'].sum()
    total_sum.name = '총사용금액'
    total_sum = total_sum.reset_index()

    df['동일공간'] = df['읍/면'] == df['가맹점 주소']
    same_sum = df.groupby(['사용자코드']).apply(lambda x : x.loc[(x['동일공간'] == True), '사용금액'].sum())
    same_sum.name = '동일공간 총사용금액'
    same_sum = same_sum.reset_index()

    result = same_sum.merge(total_sum, on = ['사용자코드'], how = 'right').fillna(0)
    result['다른지역소비비율'] = round(1 - (result['동일공간 총사용금액'] / result['총사용금액']), 4)
    return result[['사용자코드', '다른지역소비비율']]

In [158]:
##골목상권소비비율 
def corner(df1, df2):
    # df1: 총사용금액
    # df2: 원본데이터(df_0, df_1, df_2)
    e = df2.groupby(['사용자코드', '골목상권분류'])['사용금액'].sum().reset_index()
    e = e.rename(columns={'사용금액': '총 골목상권소비금액'})
    e = e.loc[e['골목상권분류'] == '골목상권']
    e = e[['사용자코드', '총 골목상권소비금액']]

    f = pd.merge(df1, e, on='사용자코드', how = 'left').fillna(0)
    f['골목상권소비비율'] = round(f['총 골목상권소비금액'] / f['총 사용금액'], 4)
    return f[['사용자코드', '골목상권소비비율']]

In [159]:
def calculate(data):
    total_amount = spend(data)                                     #df1
    region_amount = diff_region(data)                              #df2
    corner_amount = corner(total_amount, data)                     #df3
    result = total_amount.merge(region_amount, on='사용자코드', how='left').merge(corner_amount, on='사용자코드', how='left').fillna(0)
    return result

In [160]:
result0 = calculate(df_0)
result1 = calculate(df_1)
result2 = calculate(df_2)

In [161]:
dataframes = [result0, result1, result2]
for idx, df in enumerate(dataframes):
    df_head = df.head()
    print(f"Df {idx}:")
    print(df_head)

Df 0:
                           사용자코드   총 사용금액  사용빈도  1회당 평균 사용금액  다른지역소비비율  \
0     (재)부여군지역공동체활성화재단9999999991   185000    12   15416.6667    0.0000   
1           AN MEILAN01043069191  1586800    25   63472.0000    0.7941   
2   ANDAYA MERRY JANE01066493908   323600    13   24892.3077    0.5810   
3     BUNRODWATCHAREE01023240908   505320    19   26595.7895    0.0000   
4  Baik rachel soonil01099137787  6929450    57  121569.2982    0.5750   

   골목상권소비비율  
0    0.9514  
1    0.2322  
2    0.2040  
3    0.5757  
4    0.4291  
Df 1:
                        사용자코드   총 사용금액  사용빈도  1회당 평균 사용금액  다른지역소비비율  골목상권소비비율
0         (재)국제도덕협회9999990013   106080     3   35360.0000    0.0000    0.0000
1  (재)부여군지역공동체활성화재단9999999991  1086040    37   29352.4324    0.0000    0.7405
2        AN MEILAN01043069191   563200    12   46933.3333    0.8136    0.2353
3          AN YUZI01082813529   230240     7   32891.4286    0.3485    0.1650
4   BERSANOMARYANN01096096714   209110     5   41822.0000    0.4256  

In [162]:
for idx, df in enumerate(dataframes):
    na_sum = df.isna().sum()
    print(f"Df {idx}:")
    print(na_sum)

Df 0:
사용자코드          0
총 사용금액         0
사용빈도           0
1회당 평균 사용금액    0
다른지역소비비율       0
골목상권소비비율       0
dtype: int64
Df 1:
사용자코드          0
총 사용금액         0
사용빈도           0
1회당 평균 사용금액    0
다른지역소비비율       0
골목상권소비비율       0
dtype: int64
Df 2:
사용자코드          0
총 사용금액         0
사용빈도           0
1회당 평균 사용금액    0
다른지역소비비율       0
골목상권소비비율       0
dtype: int64


In [163]:
for idx, df in enumerate(dataframes):
    describe_df = df.describe()
    print(f"Df {idx}:")
    print(describe_df)

Df 0:
             총 사용금액          사용빈도   1회당 평균 사용금액      다른지역소비비율      골목상권소비비율
count  4.824600e+04  48246.000000  4.824600e+04  48246.000000  48246.000000
mean   2.187988e+06     59.255482  5.707487e+04      0.372845      0.274648
std    4.276486e+06     86.009285  1.033091e+05      0.369368      0.250832
min    1.000000e+01      1.000000  1.000000e+01      0.000000      0.000000
25%    3.192025e+05      9.000000  2.477393e+04      0.000000      0.064600
50%    9.353750e+05     25.000000  3.573465e+04      0.255500      0.221600
75%    2.390012e+06     70.000000  5.410003e+04      0.720100      0.414400
max    3.643199e+08   1245.000000  6.600000e+06      1.000000      1.000000
Df 1:
             총 사용금액          사용빈도   1회당 평균 사용금액      다른지역소비비율      골목상권소비비율
count  4.354700e+04  43547.000000  4.354700e+04  43547.000000  43547.000000
mean   2.637950e+06     72.667968  5.474033e+04      0.371722      0.281970
std    5.090448e+06    104.717352  1.098375e+05      0.375061      0.266792


## 충전데이터 

In [164]:
charge = pd.concat([charge1, charge2], axis=0)
charge

,회원구분,충전금액,발행(정책+민간),인센티브적립,인센티브사용,month,사용자코드
0,일반사용자,1000000,0,100000,0,201912,이윤주01058024525
1,가맹점주,53000,0,50,0,201912,이용인01023849506
2,일반사용자,1000000,0,100000,0,201912,김해형01073461246
3,일반사용자,500000,0,50000,0,201912,고대석01088067011
4,일반사용자,500000,0,50000,0,201912,안명환01071813783
...,...,...,...,...,...,...,...
213233,일반사용자,10000,0,1000,0,202312,박건희01068023737
213234,일반사용자,0,0,0,0,202312,정택준01064102918
213235,일반사용자,0,0,0,0,202312,김진근01054277453
213236,일반사용자,200000,0,20000,0,202312,이용규01097981551


In [165]:
#연도별 데이터프레임 나누기 
charge['month'] = pd.to_datetime(charge['month'], format='%Y%m')
year_ranges = [('2020-11', '2021-10'), ('2021-11', '2022-10'), ('2022-11', '2023-10')]
year_dict = {}
for idx, (start_date, end_date) in enumerate(year_ranges):
    start_date_dt = pd.to_datetime(start_date)
    end_date_dt = pd.to_datetime(end_date)
    data_filt = charge[(charge['month'] >= start_date_dt) & (charge['month'] <= end_date_dt)]
    year_dict[f"charge_{idx}"] = data_filt

for key, value in year_dict.items():
    globals()[f'{key}'] = value 
##charge_0, charge_1, charge_2

In [166]:
##자가충전비율 
def charge_amount(df):
    # df['사용자코드'] = df['사용자'] + df['핸드폰번호']
    # df_filt = df.loc[~((df['사용자코드'].isin(df.loc[(df['충전금액'] < 0), '사용자코드'])) | 
    #                         (df['사용자코드'].isin(df.loc[(df['발행(정책+민간)'] < 0), '사용자코드'])) |
    #                         (df['사용자코드'].isin(df.loc[(df['인센티브사용'] < 0), '사용자코드'])) |
    #                         (df['사용자코드'].isin(df.loc[(df['인센티브적립'] < 0), '사용자코드'])))]
    df_merge = df.groupby(['사용자코드'])[['충전금액', '발행(정책+민간)']].sum().reset_index()
    df_merge['자가충전비율'] = round(df_merge['충전금액'] / (df_merge['충전금액'] + df_merge['발행(정책+민간)']), 4)
    return df_merge[['사용자코드', '자가충전비율']]

In [167]:
charge_result0 = charge_amount(charge_0)
charge_result1 = charge_amount(charge_1)
charge_result2 = charge_amount(charge_2)

In [168]:
charge_df = [charge_result0, charge_result1, charge_result2]
for idx, df in enumerate(charge_df):
    describe_df = df.describe()
    print(f"Charge {idx}:")
    print(describe_df)

Charge 0:
             자가충전비율
count  59317.000000
mean       0.262351
std        0.387505
min        0.000000
25%        0.000000
50%        0.000000
75%        0.645200
max        1.000000
Charge 1:
             자가충전비율
count  47438.000000
mean       0.503777
std        0.473894
min        0.000000
25%        0.000000
50%        0.714300
75%        1.000000
max        1.000000
Charge 2:
             자가충전비율
count  50826.000000
mean       0.406394
std        0.455868
min       -0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.204900


## 사용자분석(+가맹점 분류) + 충전내역 
- 사용자분석+가맹점 = `result0`, `result1`, `result2`
- 충전내역 = `charge_result0`, `charge_result1`, `charge_result2`

In [169]:
##원 데이터에서 다시 병합할 것 
tomerge = consumer[['사용자코드', '성별', '연령대', '읍/면']].drop_duplicates()

def final(result, charge_result, tomerge):
    result_merge = result.merge(charge_result, on='사용자코드', how='left').merge(tomerge, on='사용자코드', how='left').fillna(0)
    return result_merge

In [170]:
result_merge0 = final(result0, charge_result0, tomerge)
result_merge1 = final(result1, charge_result1, tomerge)
result_merge2 = final(result2, charge_result2, tomerge)

In [171]:
result_merge0['기간'] = '2020'
result_merge1['기간'] = '2021'
result_merge2['기간'] = '2022'

result_merge = pd.concat([result_merge0, result_merge1, result_merge2], axis=0)

In [172]:
len(result_merge0) + len(result_merge1) + len(result_merge2)

128524

In [173]:
result_merge

,사용자코드,총 사용금액,사용빈도,1회당 평균 사용금액,다른지역소비비율,골목상권소비비율,자가충전비율,성별,연령대,읍/면,기간
0,(재)부여군지역공동체활성화재단9999999991,185000,12,15416.6667,0.0000,0.9514,0.0000,여자,20대,부여읍,2020
1,AN MEILAN01043069191,1586800,25,63472.0000,0.7941,0.2322,0.8000,여자,30대,홍산면,2020
2,ANDAYA MERRY JANE01066493908,323600,13,24892.3077,0.5810,0.2040,0.0000,여자,30대,기타,2020
3,BUNRODWATCHAREE01023240908,505320,19,26595.7895,0.0000,0.5757,0.0000,여자,50대,부여읍,2020
4,Baik rachel soonil01099137787,6929450,57,121569.2982,0.5750,0.4291,1.0000,여자,50대,규암면,2020
...,...,...,...,...,...,...,...,...,...,...,...
36726,황환식01094747189,1375190,39,35261.2821,0.9028,0.3863,1.0000,남자,60대,기타,2022
36727,황환택01052035416,1366535,62,22040.8871,0.8334,0.4230,1.0000,남자,60대,규암면,2022
36728,황훈희01025079842,2508306,46,54528.3913,1.0000,0.1170,1.0000,남자,50대,기타,2022
36729,황희선01074340437,1432170,31,46199.0323,0.0945,0.1187,0.5263,여자,60대,기타,2022


In [174]:
result_merge.to_csv("../Data/Preprocessed/사용자_충전_가맹점분류.csv", index=False, encoding='cp949')